In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import os
from photoz_utils import *
GB_LIMIT = 2

In [2]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1000*GB_LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
cat_data = pd.read_csv('/data/HSC/HSC_v6/CNN_visualization_v1/selected_imgs_cat.csv') # df of categorization data
not_cat_data = pd.read_csv('/data/HSC/HSC_v6/CNN_visualization_v1/selected_imgs_data.csv')

In [4]:
data = not_cat_data.merge(cat_data)

In [5]:
nn_scatter = [calculate_scatter(data['photoz_nn'][i], data['specz'], True) for i in range(len(data))]
nn_outlier = [calculate_outlier_rate(data['photoz_nn'][i], data['specz'], True) for i in range(len(data))]
nn_loss = calculate_loss(data['photoz_nn'], data['specz'])
data['scatter_nn'] = nn_scatter
data['outlier_rate_nn'] = nn_outlier
data['loss_nn'] = nn_loss

rf_scatter = [calculate_scatter(data['photoz_rf'][i], data['specz'], True) for i in range(len(data))]
rf_outlier = [calculate_outlier_rate(data['photoz_rf'][i], data['specz'], True) for i in range(len(data))]
rf_loss = calculate_loss(data['photoz_rf'], data['specz'])
data['scatter_rf'] = rf_scatter
data['outlier_rate_rf'] = rf_outlier
data['loss_rf'] = rf_loss


cnn_scatter = [calculate_scatter(data['photoz_cnn'][i], data['specz'], True) for i in range(len(data))]
cnn_outlier = [calculate_outlier_rate(data['photoz_cnn'][i], data['specz'], True) for i in range(len(data))]
cnn_loss = calculate_loss(data['photoz_cnn'], data['specz'])
data['scatter_cnn'] = cnn_scatter
data['outlier_rate_cnn'] = cnn_outlier
data['loss_cnn'] = cnn_loss

In [6]:
data.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

In [7]:
data

,index,object_id,specz,photoz_nn,bias_nn,g_cmodel_mag,g_cmodel_magsigma,r_cmodel_mag,r_cmodel_magsigma,i_cmodel_mag,...,edge_on,scatter_nn,outlier_rate_nn,loss_nn,scatter_rf,outlier_rate_rf,loss_rf,scatter_cnn,outlier_rate_cnn,loss_cnn
0,40074,70365041915877073,0.209333,2.298409,1.727462,20.939396,0.006759,20.391289,0.004909,20.199217,...,False,0.582181,0.972603,9.925165e-01,0.453882,0.972603,0.982575,0.212818,0.493151,3.913984e-04
1,16490,41579711536596015,0.094230,1.803216,1.561816,19.777716,0.001744,19.451578,0.001427,19.375889,...,True,0.494778,0.972603,9.908602e-01,0.404667,0.986301,0.981589,0.193109,0.602740,8.626186e-07
2,10864,41021550471707938,0.065730,1.515024,1.359908,20.976364,0.004788,20.924738,0.005857,20.642412,...,True,0.443911,0.972603,9.879798e-01,0.482406,0.972603,0.990892,0.188289,0.602740,4.265700e-05
3,6090,39935241573388804,0.068960,1.359743,1.207513,19.923796,0.004506,19.669414,0.005188,19.481480,...,False,0.416503,0.986301,9.848033e-01,0.456091,0.972603,0.988923,0.188649,0.602740,8.941387e-07
4,5432,39600448872667363,0.136283,1.502387,1.202257,20.204815,0.003969,20.016647,0.003717,19.693857,...,False,0.441680,0.986301,9.846722e-01,0.446191,0.972603,0.985222,0.200059,0.547945,2.753365e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,13719,41210675061614649,0.161940,0.158991,0.002538,19.813929,0.001492,19.011013,0.000910,18.552931,...,False,0.204566,0.547945,2.862763e-04,0.205639,0.547945,0.000322,0.196060,0.575342,7.927659e-02
69,7391,40146085812923356,0.299930,0.296162,0.002899,20.562515,0.002694,19.188488,0.001305,18.544563,...,False,0.228777,0.534247,3.732754e-04,0.228993,0.520548,0.000170,0.219612,0.575342,7.542526e-02
70,12656,41139473093789210,0.470460,0.469645,0.000555,22.373089,0.007589,20.648031,0.003145,19.806511,...,False,0.259398,0.534247,1.366777e-05,0.258924,0.534247,0.000252,0.269920,0.575342,6.635437e-02
71,16135,41570778004587027,0.635600,0.635401,0.000121,23.770222,0.014516,22.977875,0.015730,22.408157,...,False,0.288654,0.616438,6.552784e-07,0.288438,0.616438,0.000034,0.277651,0.561644,6.101729e-02


In [8]:
data.to_csv('/data/HSC/HSC_v6/CNN_visualization_v1/full_dataset.csv', index=False)